In [ ]:
!pip install medspacy
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.4.0/en_core_sci_sm-0.4.0.tar.gz

In [ ]:
incl_scispacy_umls_linker = False

In [ ]:
import pprint
import spacy
import medspacy
from medspacy.context import ConTextComponent
from medspacy.visualization import visualize_dep, visualize_ent

# //For linking to UMLS - not required just for NER and context annotations, only if the CUIs are required
# //Adds significantly to load time and doc processing time
if incl_scispacy_umls_linker:
    !pip install scispacy
    from scispacy.linking import EntityLinker

import warnings
warnings.filterwarnings('ignore')

pp = pprint.PrettyPrinter(indent=2)

In [ ]:
nlp = spacy.load("en_core_sci_sm")
if incl_scispacy_umls_linker:
    # //Add this pipeline component to get UMLS CUIs annotated
    nlp.add_pipe("scispacy_linker", config={"resolve_abbreviations": True, "linker_name": "umls"})
context = ConTextComponent(nlp)
nlp.add_pipe("medspacy_context")
print(nlp.pipe_names)

In [ ]:
text = """IMPRESSION: 
1. Cholelithiasis with positive sonographic Murphy sign. However gallbladder is 
nondistended without wall thickening. Equivocal for acute cholecystitis.
2. Mild intra and extrahepatic ductal dilatation. Correlate with LFTs.
If obstructive pattern, consider MRCP."""
doc = nlp(text)
sentences = [{"idx":sent_id, "start":sent.start_char, "end":sent.end_char} for sent_id, sent in enumerate(doc.sents)]
results = []
for ent in doc.ents:
    if incl_scispacy_umls_linker:
        try:
            cuis = ent._.kb_ents[0][0]
        except IndexError:
            cuis = None
    else:
        cuis = None
    result = {
        "concept": ent,
        "start": ent.start_char,
        "end": ent.end_char,
        "cui": cuis,
        "is_negated": ent._.is_negated,
        "is_uncertain": ent._.is_uncertain,
        "is_conditional": ent._.is_hypothetical,
        "is_historic": ent._.is_historical,
        "subject": "family" if ent._.is_family else "patient",
        "sentence": [sentence["idx"] for sentence in sentences if sentence["start"]<= ent.start_char and sentence["end"]>=ent.end_char][0]
    }
    results.append(result)
    if incl_scispacy_umls_linker:
        linker = nlp.get_pipe("scispacy_linker")
        for umls_ent in ent._.kb_ents:
            print(linker.kb.cui_to_entity[umls_ent[0]])

visualize_ent(doc)

In [ ]:
results

In [ ]:
for r in results:
    if r['concept'].text == "acute cholecystitis":
        pp.pprint(r)